In [13]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import osmnx as ox  
from osmnx import geocoder
import matplotlib.animation as animation
import random
import time

In [14]:
nodes = pd.read_csv('nodes.csv')
edges = pd.read_csv('edges.csv')
print(edges.head())
#fig, ax = ox.plot_graph(G)


        u           v  key       osmid   highway maxspeed              name  \
0  118725  2512504197    0  1355182986  tertiary       30  Banegårdspladsen   
1  118738  4924695592    0   699713642   service      NaN               NaN   
2  118744  1277939659    0   699713644   service      NaN               NaN   
3  118744  1277939659    1   112439593   service      NaN               NaN   
4  118744  1277939654    0   112439593   service      NaN               NaN   

   oneway reversed     length  \
0   False    False  14.315994   
1   False     True  11.838357   
2   False    False  15.868216   
3   False     True  99.070859   
4   False    False  64.339114   

                                            geometry        service bridge  \
0  LINESTRING (12.5629097 55.6739757, 12.5629715 ...            NaN    NaN   
1  LINESTRING (12.5618351 55.6628366, 12.5617904 ...            NaN    NaN   
2  LINESTRING (12.5600339 55.6617175, 12.5601929 ...  parking_aisle    NaN   
3  LINESTRING 

In [15]:
G = ox.load_graphml('graph.graphml')
print(G)

MultiDiGraph with 37113 nodes and 102980 edges


In [23]:
def plot_path(G, path):

    route_x, route_y = zip(*[(G.nodes[node]["x"], G.nodes[node]["y"]) for node in path])


    padding = 0.001 
    x_min, x_max = min(route_x) - padding, max(route_x) + padding
    y_min, y_max = min(route_y) - padding, max(route_y) + padding

    fig, ax = plt.subplots(figsize=(8, 8))

    ox.plot_graph(G, ax=ax, show=False, close=False, bgcolor='white')
    ax.plot(route_x[0], route_y[0], color='yellow', linewidth=5, marker='o')
    ax.plot(route_x[-1], route_y[-1], color='yellow', linewidth=5, marker='*')
    ax.plot(route_x, route_y, color='red')
    ax.set_xlim(x_min, x_max) 
    ax.set_ylim(y_min, y_max)
    plt.show()

def find_neighbors(G, node_id):
    ''' 
    This function returns a list with the ids of the neighbors of a node.

    Arguments:
        - G: a graph object
        - node_id: the id of the node to find the neighbors of

    Returns:
        - neighbors: a list with the ids of the neighbors of the node

    '''
    neighbors = []
    for neighbor in G.neighbors(node_id):
        neighbors.append(neighbor)

    if len(neighbors) == 0:
        print('Node has no neighbors')
    return np.array(neighbors)


def real_dist_nodes( edges, node1_id, node2_id):
    '''  
    This function returns a the distance between two connected nodes.

    Arguments:
        - edges: edges data frame
        - node1_id: node 1 id
        - node2_id: node 2 id

    Returns:
        - length: length between the connected nodes
    
    '''
    edge = edges[(edges['u'] == node1_id) & (edges['v'] == node2_id)]
    if not edge.empty:
        return edge.iloc[0]['length']
    else:
        print('Nodes are not connected')
        return None 


def euc_dist_nodes(G, edges, node1_id, node2_id):
    y1, x1, y2, x2 = [] , [], [], []

    for i in range(len(node1_id)):
        node_data = G.nodes[node1_id[i]]
        y1.append(node_data['y'])
        x1.append(node_data['x'])
    for j in range(len(node2_id)):
        node_data = G.nodes[node2_id[j]]
        y2.append(node_data['y'])
        x2.append(node_data['x'])
    y1, x1, y2, x2 = np.array(y1), np.array(x1), np.array(y2), np.array(x2)
    return ox.distance.euclidean(y1, x1, y2, x2)        
    

def distance_to_prob(dist):
    ''' 
    This function returns the probability of a path given the distance.

    Arguments:
        - dist: distance of the path

    Returns:
        - prob: normalized probability of the path

    '''
    prob = 1/dist
    return prob / np.sum(prob)


def find_path(G, edges, nodes, start_id, end_id):
    current_id = start_id
    path = [current_id]

    while current_id != end_id:
        neigh = find_neighbors(G, current_id)
        if len(neigh) == 0:
            #print('No neighbours found, terminating search.')
            break
        dist_to_end = euc_dist_nodes(G, edges, neigh, [end_id])

        if np.any(dist_to_end==0):
            #print('Path found!')
            path.append(end_id)
            break

        prob = distance_to_prob(dist_to_end)

        current_id = np.random.choice(neigh, p=prob)

        path.append(current_id)
    return np.array(path)



def populate(G, edges, nodes, start_id, end_id, num_pop):
    population= []
    for n in range(num_pop):
        path = find_path(G, edges, nodes, start_id, end_id)
        #plot_path(G,path)
        population.append(path)
    return population

        
latstart, lonstart = ox.geocoder.geocode('Nyhavn, Copenhagen, Denmark')
nodestart = ox.distance.nearest_nodes(G, lonstart, latstart)
latend, lonend = ox.geocoder.geocode('Bastard Cafe, Copenhagen, Denmark')
nodeend = ox.distance.nearest_nodes(G, lonend, latend)
path = find_path(G, edges, nodes, nodestart, nodeend)

population = populate(G, edges, nodes, nodestart, nodeend, 1000)

